In [ ]:
# !pip install prince

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from prince import FAMD

# Song Vectorization


In [ ]:
metadata = pd.read_csv("all_features.csv")
metadata = metadata.astype({"key" : str,
                            "mode" : str,
                            "time_signature" : str})
metadata = metadata.sort_values(by=["song_id"]).reset_index(drop=True)
song_order = metadata["song_id"]
metadata=metadata.drop(columns=["song_id"])
metadata.describe()

In [ ]:
set(metadata["time_signature"])

In [ ]:
famd = FAMD(n_components =64).fit(metadata)
encoded = famd.transform(metadata)
encoded.insert(0,'song_id',song_order)
encoded

In [ ]:
encoded.to_csv("song_metadata_FAMD.csv", index=False)

In [ ]:
encoded = pd.read_csv("song_metadata_FAMD.csv")
track_features = pd.read_csv("track_features.csv")
combined_df = pd.merge(encoded, track_features, on="song_id")

song_feature_dic = {}
features_numpy = combined_df.iloc[:,1:].values
for ind, row in tqdm(combined_df.iterrows(), ncols=100, total=combined_df.shape[0]):
    song_feature_dic[row["song_id"]] = [float(i) for i in list(features_numpy[ind])]

for i in tqdm(sorted(song_feature_dic), ncols=100):
    temp = np.array(song_feature_dic[i])
    temp[:64] = temp[:64]/np.linalg.norm(temp[:64])
    temp[64:] = temp[64:]/np.linalg.norm(temp[64:])
    song_feature_dic[i] = [float(i) for i in temp]

with open("/content/drive/MyDrive/Metadata/full_normalized_song_features_128.json", "w") as fp: # Change path
    json.dump(song_feature_dic, fp)

In [ ]:
encoded = pd.read_csv("song_metadata_FAMD.csv")
track_features = pd.read_csv("track_features.csv")
combined_df = pd.merge(encoded, track_features, on="song_id")

song_feature_dic = {}
features_numpy = combined_df.iloc[:,1:].values
for ind, row in tqdm(combined_df.iterrows(), ncols=100, total=combined_df.shape[0]):
    song_feature_dic[row["song_id"]] = [float(i) for i in list(features_numpy[ind])]

for i in tqdm(sorted(song_feature_dic), ncols=100):
    temp = np.array(song_feature_dic[i])
    song_feature_dic[i] = [float(i) for i in temp]

with open("/content/drive/MyDrive/Metadata/full_non_normalized_song_features_128.json", "w") as fp:
    json.dump(song_feature_dic, fp)

# User Vectorization

In [ ]:
with open("full_normalized_song_features_128.json", "r") as fp:
    song_feature_dic = json.load(fp)
triplets = pd.read_csv("filtered_triplets.csv")
user_to_song_dic = {}
for ind, row in tqdm(triplets.iterrows(), ncols=100, total=triplets.shape[0]):
    if row["user"] in user_to_song_dic:
        user_to_song_dic[row["user"]][row["song"]] = row["play_count"]
    else:
        user_to_song_dic[row["user"]] = {row["song"] : row["play_count"]}

In [ ]:
users = sorted(list(user_to_song_dic.keys()))
user_vector = {}
for user in tqdm(users, ncols=100):
    temp = np.zeros((128))
    count_songs = 0
    for song,play_count in user_to_song_dic[user].items():
        temp += play_count* np.array(song_feature_dic[song])
        count_songs += 1
    temp /= count_songs
    user_vector[user] = [float(i) for i in temp]
    
with open("weighted_normalized_sum_user_features_128.json", "w") as fp:
    json.dump(user_vector, fp)

In [ ]:
with open("/content/drive/MyDrive/NGCF_Float64/Data/MSD/train_actual.json") as fp:
    users_songs_train = json.load(fp)
users = sorted(list(user_to_song_dic.keys()))
user_vector = {}
for user in tqdm(users, ncols=100):
    temp = np.zeros((128))
    count_songs = 0
    for song,play_count in user_to_song_dic[user].items():
        if song in users_songs_train[user]:
            temp += play_count* np.array(song_feature_dic[song])
            count_songs += 1
    temp /= count_songs
    user_vector[user] = [float(i) for i in temp]
    
with open("user_features_128_train.json", "w") as fp:
    json.dump(user_vector, fp)
print(len(user_vector), len(user_vector[users[0]]))

In [ ]:
# with open("weighted_normalized_sum_user_features_128.json", "r") as fp:
#     user_vector_dic = json.load(fp)
# with open("full_normalized_song_features_128.json", "r") as fp:
#     song_feature_dic = json.load(fp)
# song_npy = np.zeros((len(song_feature_dic), 128))
# user_npy = np.zeros((len(user_vector_dic), 128))
# count = 0
# for i in tqdm(sorted(list(song_feature_dic)), ncols=100):
#     song_npy[count] = np.array(song_feature_dic[i])
#     count += 1
# count = 0
# for i in tqdm(sorted(list(user_vector_dic)), ncols=100):
#     user_npy[count] = np.array(user_vector_dic[i])
#     count += 1

In [ ]:
# np.save("user_feature_vector", user_npy)
# np.save("song_feature_vector", song_npy)

In [ ]:
# with open("user_features_128_train.json", "r") as fp:
#     user_vector_dic = json.load(fp)
# count = 0
# user_npy = np.zeros((len(user_vector_dic), 128))
# for i in tqdm(sorted(list(user_vector_dic)), ncols=100):
#     user_npy[count] = np.array(user_vector_dic[i])
#     count += 1
# np.save("user_feature_vector_train", user_npy)